In [1]:
import re
import string
import pandas as pd
from math import log
import os
import random
import re
import string
from collections import Counter
from sklearn.cluster import KMeans
import pickle

import nltk
import numpy as np
import pandas as pd

from gensim.models import Word2Vec

from nltk import word_tokenize
from nltk.tokenize import wordpunct_tokenize
from nltk.corpus import stopwords

import warnings
warnings.filterwarnings("ignore")
nltk.download("stopwords")


SEED = 42
random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
np.random.seed(SEED)

[nltk_data] Downloading package stopwords to /Users/joker/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
final_counts = []

In [5]:
def clean_text(text):
    text = str(text).lower() 
    text = text.translate (str.maketrans('', '', string.punctuation))
    text = re.sub("\d+", "", text)
    text = re.sub(r"\s+", " ", text)  # Remove multiple spaces in content
    text = re.sub('[!()-[]{};:\'"\,<>./?@#$%^&*_~–]+', '', text)
    return text

In [9]:
df = pd.read_csv(r"/Volumes/olaaaa/Thesis_backup/data/chunk0.csv", header = None, low_memory=False)
df = df[[0,21]]
df.columns = ['reg_no', 'subject_content']

In [20]:
df = df.iloc[:1000, :]

In [21]:
df['subject_content_cleaned'] = df['subject_content'].apply(clean_text)

In [22]:
df.head()

,reg_no,subject_content,subject_content_cleaned
0,-- Se/E/2015/00001,my grievance i am uploading in pdf format,my grievance i am uploading in pdf format
1,-- Se/E/2015/00002,This is in regards to Vendor Registration Appl...,this is in regards to vendor registration appl...
2,--SEL/E/2015/00001,hjjkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkk...,hjjkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkk...
3,--SEL/E/2015/00002,"Respected Sir, SUB :Telangana State - ANTI COR...",respected sir sub telangana state anti corrupt...
4,--SEL/E/2015/00003,This complaint is regarding FIR no. 347 filled...,this complaint is regarding fir no filled in g...


In [60]:
all_devnagri = [] #list to keep all the grievances with more Devnagri words than non-Devnagri


In [54]:
pref = "/Volumes/olaaaa/Thesis_backup/data/"

In [61]:
for file in os.listdir("/Volumes/olaaaa/Thesis_backup/data"):
    #print(file, "here")
    if file.endswith("csv"):
        # print("here")
        df = pd.read_csv(pref+file, header = None, low_memory=False)
        print(len(df))
        df = df[[0,21]]
        df.columns = ['reg_no', 'subject_content']
        
        df['subject_content_cleaned'] = df['subject_content'].apply(clean_text)
        tmp = [ val for val in df['subject_content_cleaned'].values \
        if len(re.findall("[\u0900-\u097F]+",val)) > len(val.split())/2]
        all_devnagri.extend(tmp)
                
        print(file)
        # break
        
        

1000001
chunk8.csv
413682
chunk9.csv
1000001
chunk2.csv
1000001
chunk3.csv
1000001
chunk1.csv
1000001
chunk0.csv
1000001
chunk4.csv
1000001
chunk5.csv
1000001
chunk7.csv
1000001
chunk6.csv


In [62]:
len(all_devnagri)

1255971

In [63]:

with open('all_devnagri.pkl', 'wb') as f:
    pickle.dump(all_devnagri, f)

In [2]:
with open('all_devnagri.pkl', 'rb') as f:
    loaded = pickle.load(f)

In [ ]:
df = pd.DataFrame(loaded)

In [ ]:
df.columns = ['subject_content']

In [ ]:
def freq(sent):
    arr = np.zeros(150)
    for letter in sent:
        if 2304 <= ord(letter) <= 2431:
            # print("here")
            arr[ord(letter) - 2304] += 1 
    #arr[::-1].sort()
    
    return arr / np.linalg.norm(arr) #normalization

In [ ]:
sent = "एटीएम मशीन का घोटाला बड़े स्तर पर किया गया है जिसमे बड़े स्तर के अधिकारी के शामिल होने की पूरी संभावना है"
for letter in sent:
    print(ord(letter))


In [ ]:
df['histograms'] = df['subject_content'].apply(freq)

In [31]:
df.isna().sum()

subject_content    0
histograms         0
dtype: int64

In [33]:
df.head()

,subject_content,histograms
0,श्री माननीय प्रधान मंत्री नरेंद्र मोदी जी मध्य...,"[0.0, 0.0, 0.1535013899613792, 0.0, 0.0, 0.015..."
1,गो माता के जीवन को सुखमय बनाने हेतु सादर निवेद...,"[0.0, 0.0, 0.05937829808056162, 0.0, 0.0, 0.02..."
2,सेबा मे माननीय प्रधानमंत्री महोदय भारत सरकार द...,"[0.0, 0.0, 0.08340845192863032, 0.0, 0.0, 0.00..."
3,प्रार्थी की ग्राम नैनपुर परगना व् तहसील मदवारा...,"[0.0, 0.0, 0.12006004503753283, 0.0, 0.0, 0.03..."
4,सेवा मैं माननीय प्रधानमंत्री महोदय विषय लोन न ...,"[0.0, 0.0, 0.16236933893402164, 0.009551137584..."


In [36]:
with open('all_devnagri_histograms.pkl', 'wb') as f:
    pickle.dump(df, f)

In [2]:
with open('all_devnagri_histograms.pkl', 'rb') as f:
    df = pickle.load(f)

In [3]:
clusters = 3
def mbkmeans_clusters(X, k):
    km = KMeans(n_clusters=k).fit(X)
    return km.labels_

In [4]:
df = df.loc[ df.histograms.map(lambda x: ~np.isnan(x).any()), :]


In [5]:
df['labels'] = mbkmeans_clusters(
    X=list(df['histograms'].values),
    k=clusters,
)

In [6]:
df.labels.value_counts()

0    477872
2    391427
1    386672
Name: labels, dtype: int64

In [14]:
df[df.labels == 1].subject_content.to_csv("tmp.csv")